<a href="https://colab.research.google.com/github/donnowhattodo/DistillationOFRoberta/blob/main/RobertaStdTech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets torch

In [1]:
!pip install transformers datasets torch

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import load_dataset

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [4]:
# Tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Convert dataset to PyTorch DataLoader
dataloader = DataLoader(dataset["train"], batch_size=32, shuffle=True)

# Teacher Model (RoBERTa Base)
teacher_model = RobertaForSequenceClassification.from_pretrained("roberta-base")
teacher_model.eval()
teacher_model.to("cuda")  # Move teacher model to GPU

# Student Model (RoBERTa Small)
student_model = RobertaForSequenceClassification.from_pretrained("roberta-base")
student_model.train()
student_model.to("cuda")  # Move student model to GPU

# Loss and hyperparameters
criterion = nn.KLDivLoss()
temperature = 3.0
alpha = 0.5
learning_rate = 1e-4
num_epochs = 5

optimizer = torch.optim.Adam(student_model.parameters(), lr=learning_rate)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Training loop
for epoch in range(num_epochs):
    total_train_loss = 0.0  # Initialize training loss for the epoch
    for batch in dataloader:
        inputs = tokenizer(batch["sentence"], padding=True, truncation=True, return_tensors="pt")
        inputs = {key: value.to("cuda") for key, value in inputs.items()}  # Move inputs to GPU
        labels = batch["label"].to("cuda")  # Move labels to GPU

        optimizer.zero_grad()

        with torch.no_grad():
            teacher_logits = teacher_model(**inputs).logits

        student_logits = student_model(**inputs).logits

        distillation_loss = criterion(
            nn.functional.log_softmax(student_logits / temperature, dim=-1),
            nn.functional.softmax(teacher_logits / temperature, dim=-1),
        )

        standard_loss = nn.functional.cross_entropy(student_logits, labels)

        total_loss = distillation_loss + alpha * standard_loss

        total_loss.backward()
        optimizer.step()

        total_train_loss += total_loss.item()

    avg_train_loss = total_train_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{num_epochs} | Average Training Loss: {avg_train_loss:.4f}")

print("Training completed.")

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2919: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch 1/5 | Average Training Loss: 0.3449
Epoch 2/5 | Average Training Loss: 0.3444
Epoch 3/5 | Average Training Loss: 0.3441
Epoch 4/5 | Average Training Loss: 0.3440
Epoch 5/5 | Average Training Loss: 0.3438
Training completed.


In [13]:
print("""

Distillation Process Steps:
  _________________________________________________________
 |                                                         |
 |    Teacher Model                Student Model           |
 |       ________                     ________             |
 |      |        |                   |        |            |
 |      |  Input |------------------>|  Input |            |
 |      |________|                   |________|            |
 |           |                             |                |
 |     Teacher Logits              Student Logits         |
 |           |                             |                |
 |           v                             v                |
 |     Distillation Loss          Standard Loss            |
 |           |                             |                |
 |           v                             v                |
 |      Total Loss                 Total Loss               |
 |           |                             |                |
 |           v                             v                |
 |    Backpropagation            Backpropagation            |
 |           |                             |                |
 |           v                             v                |
 |      Update Student           Update Student             |
 |       Parameters                Parameters              |
 |                                                         |
  _________________________________________________________


""")



Distillation Process Steps:
  _________________________________________________________
 |                                                         |
 |    Teacher Model                Student Model           |
 |       ________                     ________             |
 |      |        |                   |        |            |
 |      |  Input |------------------>|  Input |            |
 |      |________|                   |________|            |
 |           |                             |                |
 |     Teacher Logits              Student Logits         |
 |           |                             |                |
 |           v                             v                |
 |     Distillation Loss          Standard Loss            |
 |           |                             |                |
 |           v                             v                |
 |      Total Loss                 Total Loss               |
 |           |                             |       